# Defining Types of Articles

In [1]:
systematic_meta_analysis = [
    "Meta-Analysis",
    "Network Meta-Analysis",
    "Systematic Review"
    ]
   

In [2]:
clinical_trial = [
    "Adaptive Clinical Trial",
    "Clinical Study",
    "Clinical Trial",
    "Clinical Trial Protocol",
    "Clinical Trial, Phase I",
    "Clinical Trial, Phase II",
    "Clinical Trial, Phase III",
    "Clinical Trial, Phase IV",
    "Clinical Trial, Veterinary",
    "Controlled Clinical Trial",
    "Equivalence Trial",
    "Multicenter Study",
    "Pragmatic Clinical Trial",
    "Randomized Controlled Trial",
    "Randomized Controlled Trial, Veterinary"
]


In [3]:
observational_studies = [
    "Case Reports",
    "Comparative Study",
    "Evaluation Study",
    "Observational Study",
    "Observational Study, Veterinary",
    "Twin Study",
    "Validation Study",
    "Review",
    "Scientific Integrity Review",
    "Scoping Review"
]


In [4]:
# Initialize an empty list
all_study_types = []

# Extend the list with other lists
all_study_types.extend(systematic_meta_analysis)
all_study_types.extend(clinical_trial)
all_study_types.extend(observational_studies)

allowed_study_types = all_study_types

# Print the combined list
print("Total Study types:",len(all_study_types))




Total Study types: 28


# Loading Data

In [5]:
import pandas as pd

# Read Excel file
df = pd.read_excel('Data/entrez_ingredients_with_synonyms.xlsx')  # For .xls or .xlsx

# View first 5 rows
df.head()

,Root Name,PMID,Title,Article Type,Publication Year,PubMed URL,Search Term(s)
0,Acetyl L-Carnitine,10022226,Micronutrients prevent cancer and delay aging.,"Journal Article; Research Support, Non-U.S. Go...",1998,https://pubmed.ncbi.nlm.nih.gov/10022226/,Carnitine
1,Acetyl L-Carnitine,10030388,Significance of skeletal muscle properties on ...,Comparative Study; Journal Article; Research S...,1999,https://pubmed.ncbi.nlm.nih.gov/10030388/,Carnitine
2,Acetyl L-Carnitine,10036643,Cellular dysmetabolism: the dark side of HIV-1...,Journal Article; Review,1996,https://pubmed.ncbi.nlm.nih.gov/10036643/,Carnitine
3,Acetyl L-Carnitine,10052020,Substrate utilization and work efficiency duri...,Clinical Trial; Controlled Clinical Trial; Jou...,1999,https://pubmed.ncbi.nlm.nih.gov/10052020/,Carnitine
4,Acetyl L-Carnitine,10067662,L-carnitine improves glucose disposal in type ...,Clinical Trial; Controlled Clinical Trial; Jou...,1999,https://pubmed.ncbi.nlm.nih.gov/10067662/,Carnitine


In [6]:
df.shape

(277397, 7)

# Adding Synonyms

In [6]:
import numpy as np

synonyms_df = pd.read_excel("76 Ingredient List.xlsx")
synonyms_df['Common Name(s)'] = synonyms_df['Common Name(s)'].astype(str).str.replace(',', ';')
synonyms_df['Other Search Words'] = synonyms_df['Other Search Words'].astype(str).str.replace(',', ';')
synonyms_df = synonyms_df.replace(['nan', np.nan], '') 
synonyms_df['Synonyms'] = (
    synonyms_df['Common Name(s)'].fillna('')
    .str.strip() + '; ' +
    synonyms_df['Scientific Name(s)'].fillna('')
    .str.strip() + '; ' +
    synonyms_df['Other Search Words'].fillna('')
    .str.strip()
)
synonyms_df.head()

,Ingredient,Common Name(s),Scientific Name(s),Other Search Words,Synonyms
0,Ashwagandha,Ashwagandha; Indian Ginseng; Winter Cherry,Withania somnifera,Withanolide; Withanolides,Ashwagandha; Indian Ginseng; Winter Cherry; Wi...
1,Chamomile,Chamomile; English Chamomile; German Chamomile...,Matricaria chamomilla; Chamaemelum nobile; Mat...,,Chamomile; English Chamomile; German Chamomile...
2,GABA,GABA,Gamma-Aminobutyric Acid,,GABA; Gamma-Aminobutyric Acid;
3,Glycine,Glycine; L-Glycine,Aminoacetic Acid,,Glycine; L-Glycine; Aminoacetic Acid;
4,Lemon Balm,Melissa; Balm Mint; Lemon Balm Extract,Melissa officinalis,,Melissa; Balm Mint; Lemon Balm Extract; Meliss...


In [7]:
df_merged = df.merge(
    synonyms_df[['Ingredient', 'Synonyms']],  # only the columns we need
    left_on='Root Name',               # key in df
    right_on='Ingredient',             # key in synonyms_df
    how='left'                         # keep all rows from df
)

# Drop duplicate 'Ingredient' column
df = df_merged.drop(columns=['Ingredient'])
df.head()

,Root Name,PMID,Title,Article Type,Publication Year,PubMed URL,Search Term(s),Synonyms
0,Acetyl L-Carnitine,10022226,Micronutrients prevent cancer and delay aging.,"Journal Article; Research Support, Non-U.S. Go...",1998,https://pubmed.ncbi.nlm.nih.gov/10022226/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...
1,Acetyl L-Carnitine,10030388,Significance of skeletal muscle properties on ...,Comparative Study; Journal Article; Research S...,1999,https://pubmed.ncbi.nlm.nih.gov/10030388/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...
2,Acetyl L-Carnitine,10036643,Cellular dysmetabolism: the dark side of HIV-1...,Journal Article; Review,1996,https://pubmed.ncbi.nlm.nih.gov/10036643/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...
3,Acetyl L-Carnitine,10052020,Substrate utilization and work efficiency duri...,Clinical Trial; Controlled Clinical Trial; Jou...,1999,https://pubmed.ncbi.nlm.nih.gov/10052020/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...
4,Acetyl L-Carnitine,10067662,L-carnitine improves glucose disposal in type ...,Clinical Trial; Controlled Clinical Trial; Jou...,1999,https://pubmed.ncbi.nlm.nih.gov/10067662/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...


# Defining Neuraci Type

In [8]:
# Define mapping of NeuraciTypes
priority_mapping = [
    (systematic_meta_analysis,'Systematic Review & Meta Analysis'), # 1st Priority
    (clinical_trial, 'Clinical Trials'),                            # 2nd Priority
    (observational_studies, 'Observational Studies & Reviews'),     # 3rd Priority

]

# Function to assign NeuraciType based on priority
def classify_type(type_list):
    for reference_list, category in priority_mapping:
        if any(item in reference_list for item in type_list):
            return category
    return 'Unclassified'  # optional, in case nothing matches

# Function to assign NeuraciType based on priority and return the matched item
# def classify_type(type_list):
#     for reference_list, category in priority_mapping:
#         for item in type_list:
#             if item in reference_list:
#                 return category, item  # Return both category and matched item
#     return 'Unclassified', None  # Default if nothing matches


# Create new column with a list of article types
df['Type_list'] = df['Article Type'].apply(lambda x: set([item.strip() for item in x.split(';')]))


# Apply function to create new column
#df['PubMedType'] = df['Type_list'].apply(lambda x: x.intersection(set(allowed_study_types)))
df['PubMedType'] = df['Type_list'].apply(
    lambda x: ', '.join(x.intersection(set(allowed_study_types))) if isinstance(x, set) else ''
)
df["NeuraciType"] = df['Type_list'].apply(lambda x: pd.Series(classify_type(x)))

df.head()

,Root Name,PMID,Title,Article Type,Publication Year,PubMed URL,Search Term(s),Synonyms,Type_list,PubMedType,NeuraciType
0,Acetyl L-Carnitine,10022226,Micronutrients prevent cancer and delay aging.,"Journal Article; Research Support, Non-U.S. Go...",1998,https://pubmed.ncbi.nlm.nih.gov/10022226/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...,"{Journal Article, Review, Research Support, No...",Review,Observational Studies & Reviews
1,Acetyl L-Carnitine,10030388,Significance of skeletal muscle properties on ...,Comparative Study; Journal Article; Research S...,1999,https://pubmed.ncbi.nlm.nih.gov/10030388/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...,"{Journal Article, Comparative Study, Research ...",Comparative Study,Observational Studies & Reviews
2,Acetyl L-Carnitine,10036643,Cellular dysmetabolism: the dark side of HIV-1...,Journal Article; Review,1996,https://pubmed.ncbi.nlm.nih.gov/10036643/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...,"{Journal Article, Review}",Review,Observational Studies & Reviews
3,Acetyl L-Carnitine,10052020,Substrate utilization and work efficiency duri...,Clinical Trial; Controlled Clinical Trial; Jou...,1999,https://pubmed.ncbi.nlm.nih.gov/10052020/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...,"{Journal Article, Research Support, Non-U.S. G...","Controlled Clinical Trial, Clinical Trial",Clinical Trials
4,Acetyl L-Carnitine,10067662,L-carnitine improves glucose disposal in type ...,Clinical Trial; Controlled Clinical Trial; Jou...,1999,https://pubmed.ncbi.nlm.nih.gov/10067662/,Carnitine,Carnitine; ALCAR; Acetylcarnitine; Acetyl L Ca...,"{Journal Article, Controlled Clinical Trial, C...","Controlled Clinical Trial, Clinical Trial",Clinical Trials


In [10]:
ingredients = ["Cedarwood"]


In [20]:
from UPDATED_meta_data_generation import process_pmids

#ingredient_df = df[df['Root Name'].isin([10473175])]
ingredient_df = df[df['Root Name'].isin(ingredients)]
#ingredient_df = df[(df['Root Name'].isin(ingredients)) & (df['NeuraciType'] == 'Clinical Trials')]
#ingredient_df = duplicates_df[duplicates_df['PMID'].isin(pmids)]
process_pmids(ingredient_df['Root Name'],ingredient_df['Search Term(s)'],ingredient_df['Synonyms'],ingredient_df['PMID'], ingredient_df['PubMedType'],output_file="cedarwood_10|7.json") 

Processing PMIDs:   0%|          | 0/97 [00:00<?, ?pmid/s]

Synonyms Cedarwood; Cedrus; 


Processing PMIDs:   1%|          | 1/97 [00:07<11:31,  7.21s/pmid]

Synonyms Cedarwood; Cedrus; 


Processing PMIDs:   2%|▏         | 2/97 [00:12<09:12,  5.82s/pmid]

...

Synonyms Cedarwood; Cedrus; 


Processing PMIDs:  99%|█████████▉| 96/97 [09:47<00:04,  4.02s/pmid]

Synonyms Cedarwood; Cedrus; 


Processing PMIDs: 100%|██████████| 97/97 [09:51<00:00,  6.10s/pmid]


[{'root_name': 'Cedarwood',
  'search_term': 'Cedrus',
  'synonyms': 'Cedarwood; Cedrus; ',
  'PMID': 10473175,
  'pubmed_type': 'Comparative Study',
  'metadata': {'duration_days': None,
   'sample_size': None,
   'sample_gender': ['unspecified'],
   'species': ['other'],
   'population': 'turkish plants used in folk medicine for gastric ailments',
   'study_type': ['unspecified'],
   'focus': ['secondary'],
   'benefits': ['anti-helicobacter pylori effect'],
   'synergies_interactions_positive': [],
   'synergies_interactions_negative': [],
   'safety_side_effects': [],
   'interventions': [{'ingredient': 'cedrus libani',
     'daily_dosage': None,
     'units': '',
     'original_text': 'cones of cedrus libani'}],
   'usage': ['unspecified'],
   'conditions': ['gastric ailments', 'peptic ulcers'],
   'biomarkers': [],
   'functions': [],
   'purpose': 'to study the anti-helicobacter pylori effect of extracts from turkish plants used in folk medicine for gastric ailments.',
   'concl

# Enriching Data

In [23]:
import json

ingredient_df_lookup = ingredient_df.set_index(["PMID",'Root Name'])[["Publication Year","NeuraciType","Synonyms","PubMed URL"]].to_dict(orient="index")


# Read JSON file
with open("cedarwood_10|7.json", "r", encoding="utf-8") as f:
    json_list = json.load(f)

# Add attributes 
for record in json_list:
    pmid = record["PMID"]
    root_name = record['root_name']
    key = (pmid,root_name)
    if key in ingredient_df_lookup:
        lookup_data = ingredient_df_lookup[key]
        record.setdefault("metadata", {})  # ensure "metadata" exists
        record["metadata"]["published_year"] = lookup_data["Publication Year"]
        record["metadata"]["neuraci_type"] = lookup_data["NeuraciType"]
        #record["metadata"]["synonyms"] = lookup_data["Synonyms"]
        record["metadata"]["url"] = lookup_data["PubMed URL"]

# Save JSON file
with open("cedarwood_modified_enriched.json", "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=2, ensure_ascii=False)

# Converting into Structured Data

In [24]:
import json
import pandas as pd

# # --- Step 1: Load JSON file ---
# with open("data.json", "r", encoding="utf-8") as f:
#     records = json.load(f)   # assuming it's a list of JSON objects

rows = []
records = json_list
# --- Step 2: Flatten each record ---
for rec in records:
    pmid = rec.get("PMID")
    root_name = rec.get("root_name")
    search_term = rec.get("search_term")
    meta = rec.get("metadata", {})
    pubmed_type = rec.get("pubmed_type")
    synonyms = rec.get('synonyms')

    row = {
        "root_name":root_name,
        "Search Term(s)":search_term,
        "PMID": pmid,
        "synonyms":synonyms,
        "published_year": meta.get("published_year"),
        "neuraci_type": meta.get("neuraci_type"),
        "pubmed_type":pubmed_type,
        "study_type": ";".join(meta.get('study_type')),
        "duration_days": meta.get("duration_days"),
        "sample_size": meta.get("sample_size"),
        "sample_gender": ";".join(meta.get("sample_gender", [])),
        "species": ";".join(meta.get("species", [])),
        "population": meta.get("population"),
        "purpose": meta.get("purpose"),
        "focus": "".join(meta.get("focus")),
        "benefits": ";".join(meta.get("benefits", [])),
        "synergies_interactions_positive": ";".join(meta.get("synergies_interactions_positive", [])),
        "synergies_interactions_negative": ";".join(meta.get("synergies_interactions_negative", [])),
        "safety_side_effects": ";".join(meta.get("safety_side_effects", [])),
        "conditions": ";".join(meta.get("conditions", [])),
        "biomarkers": ";".join(meta.get("biomarkers", [])),
        "symptoms": ";".join(meta.get("symptoms", [])),
        "keywords": ";".join(meta.get("keywords", [])),
        "diseases": ";".join(meta.get("diseases", [])),
        "mechanism": ";".join(meta.get("mechanism", [])),
        "usage": ";".join(meta.get("usage", [])),
        "conclusion":meta.get("conclusion"),
        "url": meta.get("url", []),
    }

    # --- Expand interventions ---
    for i, iv in enumerate(meta.get("interventions", []), start=1):
        row[f"intervention{i}_ingredient"] = iv.get("ingredient")
        row[f"intervention{i}_daily_dosage"] = iv.get("daily_dosage")
        row[f"intervention{i}_units"] = iv.get("units")
        row[f"intervention{i}_original_text"] = iv.get("original_text")

    # --- Expand outcomes ---
    for j, oc in enumerate(meta.get("outcomes", []), start=1):
        row[f"outcome{j}_name"] = oc.get("name")
        row[f"outcome{j}_domain"] = oc.get("domain")
        row[f"outcome{j}_type"] = oc.get("type")
        row[f"outcome{j}_result"] = oc.get("result")

    rows.append(row)

# --- Step 3: Convert to DataFrame ---
output_df = pd.DataFrame(rows)

# --- Step 4: Save to CSV ---
output_df.to_csv("Cedarwood_modified_output.csv", index=False)
output_df.head()

,root_name,Search Term(s),PMID,synonyms,published_year,neuraci_type,pubmed_type,study_type,duration_days,sample_size,...,outcome17_type,outcome17_result,outcome18_name,outcome18_domain,outcome18_type,outcome18_result,outcome19_name,outcome19_domain,outcome19_type,outcome19_result
0,Cedarwood,Cedrus,10473175,Cedarwood; Cedrus;,1999,Observational Studies & Reviews,Comparative Study,unspecified,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cedarwood,Cedrus,11524119,Cedarwood; Cedrus;,2001,Observational Studies & Reviews,Comparative Study,unspecified,NaN,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cedarwood,Cedrus,11876259,Cedarwood; Cedrus;,2002,Observational Studies & Reviews,"Comparative Study, Review",review,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cedarwood,Cedrus,11876600,Cedarwood; Cedrus;,2002,Clinical Trials,"Controlled Clinical Trial, Clinical Trial",non-randomized trial,730.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cedarwood,Cedrus,11896972,Cedarwood; Cedrus;,2002,Observational Studies & Reviews,Case Reports,unspecified,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pd.set_option('display.max_columns', None)

In [26]:
output_df.head()

,root_name,Search Term(s),PMID,synonyms,published_year,neuraci_type,pubmed_type,study_type,duration_days,sample_size,sample_gender,species,population,purpose,focus,benefits,synergies_interactions_positive,synergies_interactions_negative,safety_side_effects,conditions,biomarkers,symptoms,keywords,diseases,mechanism,usage,conclusion,url,intervention1_ingredient,intervention1_daily_dosage,intervention1_units,intervention1_original_text,outcome1_name,outcome1_domain,outcome1_type,outcome1_result,outcome2_name,outcome2_domain,outcome2_type,outcome2_result,outcome3_name,outcome3_domain,outcome3_type,outcome3_result,outcome4_name,outcome4_domain,outcome4_type,outcome4_result,outcome5_name,outcome5_domain,outcome5_type,outcome5_result,outcome6_name,outcome6_domain,outcome6_type,outcome6_result,outcome7_name,outcome7_domain,outcome7_type,outcome7_result,intervention2_ingredient,intervention2_daily_dosage,intervention2_units,intervention2_original_text,outcome8_name,outcome8_domain,outcome8_type,outcome8_result,outcome9_name,outcome9_domain,outcome9_type,outcome9_result,intervention3_ingredient,intervention3_daily_dosage,intervention3_units,intervention3_original_text,intervention4_ingredient,intervention4_daily_dosage,intervention4_units,intervention4_original_text,intervention5_ingredient,intervention5_daily_dosage,intervention5_units,intervention5_original_text,outcome10_name,outcome10_domain,outcome10_type,outcome10_result,outcome11_name,outcome11_domain,outcome11_type,outcome11_result,outcome12_name,outcome12_domain,outcome12_type,outcome12_result,outcome13_name,outcome13_domain,outcome13_type,outcome13_result,outcome14_name,outcome14_domain,outcome14_type,outcome14_result,outcome15_name,outcome15_domain,outcome15_type,outcome15_result,outcome16_name,outcome16_domain,outcome16_type,outcome16_result,outcome17_name,outcome17_domain,outcome17_type,outcome17_result,outcome18_name,outcome18_domain,outcome18_type,outcome18_result,outcome19_name,outcome19_domain,outcome19_type,outcome19_result
0,Cedarwood,Cedrus,10473175,Cedarwood; Cedrus;,1999,Observational Studies & Reviews,Comparative Study,unspecified,NaN,NaN,unspecified,other,turkish plants used in folk medicine for gastr...,to study the anti-helicobacter pylori effect o...,secondary,anti-helicobacter pylori effect,,,,gastric ailments;peptic ulcers,,,anti-helicobacter pylori;turkish plants;folk m...,helicobacter pylori infection,,unspecified,the study found that extracts from six turkish...,https://pubmed.ncbi.nlm.nih.gov/10473175/,cedrus libani,NaN,,cones of cedrus libani,anti-helicobacter pylori effect,condition,primary,improved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cedarwood,Cedrus,11524119,Cedarwood; Cedrus;,2001,Observational Studies & Reviews,Comparative Study,unspecified,NaN,137.0,unspecified,other,137 species of gymnosperms belonging to 14 fam...,to determine the fatty acid composition of pho...,secondary,,,,,,fatty acid composition;chloroplastic lipids;di...,,gymnosperms;fatty acid composition;cedrus;pina...,,,unspecified,"a clear-cut separation of the genera abies, la...",https://pubmed.ncbi.nlm.nih.gov/11524119/,NaN,NaN,NaN,NaN,fatty acid composition,biomarker,primary,not_reported,chloroplastic lipids,biomarker,secondary,not_reported,digalactosyldiacylglycerol,biomarker,secondary,not_reported,monogalactosyldiacylglycerol,biomarker,secondary,not_reported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cedarwood